# Análise Nodal

### Lendo as informações de Circuito a partir de um arquivo que mostra sua configuração topológica tipo GRAFO

In [15]:
import numpy as np
import csv

file = open('dados/circuito.csv', newline='')
reader = csv.reader(file)
header = next(reader)
chaves = header[0].split(';')
print(chaves)

for line in reader:
    print(line[0])

['RAMO', 'NP', 'NC', 'TIPO', 'VALOR']
1;1;0;I;2
2;1;0;R;5
3;1;2;R;2
4;2;0;R;3


In [3]:
print(str(header))

['RAMO;NP;NC;TIPO;VALOR']


In [4]:
with open('dados/circuito.csv') as csvfile:
   reader = csv.DictReader(csvfile)
   for row in reader:
       print(tuple(row.values()))

('1;1;0;I;2',)
('2;1;0;R;5',)
('3;1;2;R;2',)
('4;2;0;R;3',)


In [5]:
import csv

arquivo = open('dados/circuito.csv')

linhas = csv.reader(arquivo)

for linha in linhas:
    print(linha)
            

['RAMO;NP;NC;TIPO;VALOR']
['1;1;0;I;2']
['2;1;0;R;5']
['3;1;2;R;2']
['4;2;0;R;3']


In [30]:
import csv
import numpy as np

arquivo = open('dados/circuito.csv')

dados = csv.DictReader(arquivo, delimiter =';')
circuito = {}
nos = [] # lista que tera elementos duplicados removidos

for d in dados:
    #print (d)
    #print (d["RAMO"] )
    ramo = {}
    ramo['NP'] = d['NP']
    ramo['NC'] = d['NC']
    ramo['TIPO'] = d['TIPO']
    ramo['VALOR'] = d['VALOR']
    ramo['TENSAO'] = 0
    ramo['CORRENTE'] = 0
    
    nos.append(d['NP'])
    nos.append(d['NC'])
    
    circuito[d['RAMO']] = ramo
ramos = list(circuito.keys())
nos = set(nos)
nos = list(nos)

# numero de Nós e Ramos do Circuito
nN = len(nos) - 1 # O nó 0 é o GND
nR = len(ramos)

# A Matriz Transição Nós-Ramos
A = np.zeros((nN, nR))
# Is Vetor Fonte de Corrente de ramo
Is = np.zeros((nR,1))
# Vs Vetor Fonte de Tensão de ramo
Vs = np.zeros((nR,1))
# Condutâncias de ramos
G = np.zeros((nR,nR))
G[0,0] = 1e-10  # baixa condutância -- alta resistência
G[1,1]= 1./5
G[2,2]= 1./2
G[3,3] = 1./3

for r in ramos:
    # Formação de A
    vN = int(circuito[r]['NP'])
    if vN > 0:
        A[vN - 1, int(r) - 1] = 1
    vN = int(circuito[r]['NC'])
    if vN > 0:
        A[vN - 1, int(r) - 1] = -1
    # Tipo de Ramo
    vR = int(r) - 1
    tipo = circuito[r]['TIPO']
    valor = float(circuito[r]['VALOR'])
    if tipo == 'I':
        Is[vR, 0] = valor
        G[vR,vR] = 1e-10
    elif tipo == 'V':
        Vs[vR, 0] = valor
        G[vR,vR] = 1e10
    elif tipo == "R":
        G[vR,vR] = 1./valor
    elif tipo == "G":
        G[vR,vR] = valor

print(" G : Condutâncias de Ramos \n", G)
Is = np.matrix(Is)
Vs = np.matrix(Vs)
G = np.matrix(G)
A = np.matrix(A)

Gbarra = A*G*A.T
print(" Gbarra : Condutâncias de Nós \n", Gbarra)
E = Gbarra.I*A*(G*Vs-Is)
print(" E : Tensão Nodal \n", E)
V = A.T*E
print(" V : Tensão em cada Ramo \n", V)
I=Is+G*(V-Vs)
print(" I : Corrente em cada Ramo \n", I)
for i in range(nR):
    circuito[str(i+1)]['TENSAO'] = float(V[i])
    circuito[str(i+1)]['CORRENTE'] = float(I[i])


 G : Condutâncias de Ramos 
 [[1.00000000e-10 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.00000000e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00000000e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.33333333e-01]]
 Gbarra : Condutâncias de Nós 
 [[ 0.7        -0.5       ]
 [-0.5         0.83333333]]
 E : Tensão Nodal 
 [[5.]
 [3.]]
 V : Tensão em cada Ramo 
 [[5.]
 [5.]
 [2.]
 [3.]]
 I : Corrente em cada Ramo 
 [[-2.]
 [ 1.]
 [ 1.]
 [ 1.]]


In [28]:
nR

4

In [1]:
import csv
import numpy as np

def grafoCircuito(nomeArq):
    nomeArq = 'dados/' + nomeArq + '.csv'
    arquivo = open(nomeArq)

    dados = csv.DictReader(arquivo, delimiter =';')
    circuito = {}


    for d in dados:
        #print (d)
        #print (d["RAMO"] )
        ramo = {}
        ramo['NP'] = d['NP']
        ramo['NC'] = d['NC']
        ramo['TIPO'] = d['TIPO']
        ramo['VALOR'] = d['VALOR']
        ramo['TENSAO'] = 0
        ramo['CORRENTE'] = 0
    
        circuito[d['RAMO']] = ramo
        
    return circuito


def matrizes(circuito):
    ramos = list(circuito.keys())
    nos = [circuito[k]['NP'] for k in circuito.keys() ]
    nos += [circuito[k]['NC'] for k in circuito.keys() ]
    nos = set(nos)
    nos = list(nos)
    # numero de Nós e Ramos do Circuito
    nN = len(nos) - 1 # O nó 0 é o GND
    nR = len(ramos)

    # A Matriz Transição Nós-Ramos
    A = np.zeros((nN, nR))
    # Is Vetor Fonte de Corrente de ramo
    Is = np.zeros((nR,1))
    # Vs Vetor Fonte de Tensão de ramo
    Vs = np.zeros((nR,1))
    # Condutâncias de ramos
    G = np.zeros((nR,nR))

    for r in ramos:
        # Formação de A
        vN = int(circuito[r]['NP'])
        if vN > 0:
            A[vN - 1, int(r) - 1] = 1
        vN = int(circuito[r]['NC'])
        if vN > 0:
            A[vN - 1, int(r) - 1] = -1
        # Tipo de Ramo
        vR = int(r) - 1
        tipo = circuito[r]['TIPO']
        valor = float(circuito[r]['VALOR'])
        if tipo == 'I':
            Is[vR, 0] = valor
            G[vR,vR] = 1e-10
        elif tipo == 'V':
            Vs[vR, 0] = valor
            G[vR,vR] = 1e10
        elif tipo == "R":
            G[vR,vR] = 1./valor
        elif tipo == "G":
            G[vR,vR] = valor

    print(" A : Matriz Incidência Nó-Ramo \n", A)
    print(" G : Condutâncias de Ramos \n", G)
    Is = np.matrix(Is)
    Vs = np.matrix(Vs)
    G = np.matrix(G)
    A = np.matrix(A)

    Gbarra = A*G*A.T
    print(" Gbarra : Condutâncias de Nós \n", Gbarra)
    Inodal = A*(G*Vs-Is)
    print(" Inodal : Fonte de Corrente no Nó \n", Inodal)
    E = Gbarra.I*Inodal
    print(" E : Tensão Nodal \n", E)
    V = A.T*E
    print(" V : Tensão em cada Ramo \n", V)
    I=Is+G*(V-Vs)
    print(" I : Corrente em cada Ramo \n", I)
    #for i in range(nR):
    #    circuito[str(i+1)]['TENSAO'] = float(V[i])
    #    circuito[str(i+1)]['CORRENTE'] = float(I[i])

In [2]:
circ = grafoCircuito('divisor')
matrizes(circ)

 A : Matriz Incidência Nó-Ramo 
 [[ 1.  0.  1.]
 [-1.  1.  0.]]
 G : Condutâncias de Ramos 
 [[5.00000000e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.33333333e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+10]]
 Gbarra : Condutâncias de Nós 
 [[ 1.00000000e+10 -5.00000000e-01]
 [-5.00000000e-01  8.33333333e-01]]
 Inodal : Fonte de Corrente no Nó 
 [[1.e+11]
 [0.e+00]]
 E : Tensão Nodal 
 [[10.]
 [ 6.]]
 V : Tensão em cada Ramo 
 [[ 4.]
 [ 6.]
 [10.]]
 I : Corrente em cada Ramo 
 [[ 2.        ]
 [ 2.        ]
 [-2.00000017]]


In [5]:
circ = grafoCircuito('divisorC')
matrizes(circ)

 A : Matriz Incidência Nó-Ramo 
 [[1. 1. 1.]]
 G : Condutâncias de Ramos 
 [[5.00000000e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.33333333e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e-10]]
 Gbarra : Condutâncias de Nós 
 [[0.83333333]]
 Inodal : Fonte de Corrente no Nó 
 [[5.]]
 E : Tensão Nodal 
 [[6.]]
 V : Tensão em cada Ramo 
 [[6.]
 [6.]
 [6.]]
 I : Corrente em cada Ramo 
 [[ 3.]
 [ 2.]
 [-5.]]


In [32]:
circ = grafoCircuito('circuito')
matrizes(circ)

 G : Condutâncias de Ramos 
 [[1.00000000e-10 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.00000000e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.00000000e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.33333333e-01]]
 Gbarra : Condutâncias de Nós 
 [[ 0.7        -0.5       ]
 [-0.5         0.83333333]]
 E : Tensão Nodal 
 [[5.]
 [3.]]
 V : Tensão em cada Ramo 
 [[5.]
 [5.]
 [2.]
 [3.]]
 I : Corrente em cada Ramo 
 [[-2.]
 [ 1.]
 [ 1.]
 [ 1.]]


In [13]:
nos = []
for k in circ.keys():
    nos.append(circ[k]['NP'])
    nos.append(circ[k]['NC'])
nos = set(nos)
nos = list(nos)
nos

['1', '0', '2']

['1', '0', '2']

In [15]:
nos

['1', '1', '1', '2']

In [10]:
circ['1']['NP']

'1'

In [37]:
circuito

{'1': {'NP': '1',
  'NC': '0',
  'TIPO': 'I',
  'VALOR': '-2',
  'TENSAO': 4.999999998750001,
  'CORRENTE': -1.9999999995},
 '2': {'NP': '1',
  'NC': '0',
  'TIPO': 'R',
  'VALOR': '5',
  'TENSAO': 4.999999998750001,
  'CORRENTE': 0.9999999997500002},
 '3': {'NP': '1',
  'NC': '2',
  'TIPO': 'R',
  'VALOR': '2',
  'TENSAO': 1.9999999995000004,
  'CORRENTE': 0.9999999997500002},
 '4': {'NP': '2',
  'NC': '0',
  'TIPO': 'R',
  'VALOR': '3',
  'TENSAO': 2.9999999992500004,
  'CORRENTE': 0.9999999997500001}}

In [45]:
circuito

{'1': {'NP': '1', 'NC': '0', 'TIPO': 'I', 'VALOR': '2'},
 '2': {'NP': '1', 'NC': '0', 'TIPO': 'R', 'VALOR': '5'},
 '3': {'NP': '1', 'NC': '2', 'TIPO': 'R', 'VALOR': '2'},
 '4': {'NP': '2', 'NC': '0', 'TIPO': 'R', 'VALOR': '3'}}

In [46]:

ramos

['1', '2', '3', '4']

In [47]:

nos

['0', '1', '2']

In [49]:
circuito['1']['NP']

'1'

In [11]:
5/6

0.8333333333333334